In [2]:
import os

os.environ['SPARK_HOME'] = "C:\spark"


In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [6]:
from pyspark.sql import functions as f

In [7]:
produtos = spark.createDataFrame(
    [
        ('1', 'Bebidas', 'Água mineral'), 
        ('2', 'Limpeza', 'Sabão em pó'), 
        ('3', 'Frios', 'Queijo'), 
        ('4', 'Bebidas', 'Refrigerante'),
        ('5', 'Pet', 'Ração para cães')
    ],
    ['id', 'cat', 'prod']
)

impostos = spark.createDataFrame(
    [
        ('Bebidas', 0.15), 
        ('Limpeza', 0.05),
        ('Frios', 0.065),
        ('Carnes', 0.08)
    ],
    ['cat', 'tax']
)

In [8]:
produtos.toPandas()

,id,cat,prod
0,1,Bebidas,Água mineral
1,2,Limpeza,Sabão em pó
2,3,Frios,Queijo
3,4,Bebidas,Refrigerante
4,5,Pet,Ração para cães


In [9]:
impostos.toPandas()

,cat,tax
0,Bebidas,0.150
1,Limpeza,0.050
2,Frios,0.065
3,Carnes,0.080


In [10]:
#remove das duas tabelas o que ele não acha ligação
produtos.join(impostos, 'cat', how='inner')\
.sort('id').show()

+-------+---+------------+-----+
|    cat| id|        prod|  tax|
+-------+---+------------+-----+
|Bebidas|  1|Água mineral| 0.15|
|Limpeza|  2| Sabão em pó| 0.05|
|  Frios|  3|      Queijo|0.065|
|Bebidas|  4|Refrigerante| 0.15|
+-------+---+------------+-----+



In [11]:
#left remove da segunda tabela

produtos.join(impostos, 'cat', how='left').sort('id').show()

+-------+---+---------------+-----+
|    cat| id|           prod|  tax|
+-------+---+---------------+-----+
|Bebidas|  1|   Água mineral| 0.15|
|Limpeza|  2|    Sabão em pó| 0.05|
|  Frios|  3|         Queijo|0.065|
|Bebidas|  4|   Refrigerante| 0.15|
|    Pet|  5|Ração para cães| NULL|
+-------+---+---------------+-----+



In [12]:
#junta todas as colunas das duas tabelas
produtos.join(impostos, 'cat', how='outer').show()

+-------+----+---------------+-----+
|    cat|  id|           prod|  tax|
+-------+----+---------------+-----+
|Bebidas|   1|   Água mineral| 0.15|
|Bebidas|   4|   Refrigerante| 0.15|
| Carnes|NULL|           NULL| 0.08|
|  Frios|   3|         Queijo|0.065|
|Limpeza|   2|    Sabão em pó| 0.05|
|    Pet|   5|Ração para cães| NULL|
+-------+----+---------------+-----+



In [15]:
df = produtos.join(impostos, 'cat', how='right')\
    .sort('id')

In [16]:
df.withColumnRenamed('cat', 'categoria').show()

+---------+----+------------+-----+
|categoria|  id|        prod|  tax|
+---------+----+------------+-----+
|   Carnes|NULL|        NULL| 0.08|
|  Bebidas|   1|Água mineral| 0.15|
|  Limpeza|   2| Sabão em pó| 0.05|
|    Frios|   3|      Queijo|0.065|
|  Bebidas|   4|Refrigerante| 0.15|
+---------+----+------------+-----+

